In [1]:
import numpy as np
import pandas as pd
import random
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from joblib import dump
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import itertools
from sklearn.utils import shuffle
from scipy import signal
%matplotlib inline


from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectFdr, chi2

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

from utils.svm import preProcess, evaluate_set
from utils.visualize import showMe
from utils.augment import apply_augment
from config.default import *


%load_ext autoreload
%autoreload 2


C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def create_labels(X):
    y = []
    for i, r in enumerate(X):
        l = np.ones(X[r].shape[0])*i
        y = y + l.tolist()
    y = np.array(y)
    return y


In [ ]:
root_path = 'C:/resources/EMG/'
post_fix = '_1s_cleaned' #'_1s_new' #
classes = settings['classes']
subjects = os.listdir(root_path)
results = {} #dictionary of pandas dataframes
cv_results = {}
for subject in subjects:
    print(f'Cross validating, leaving out subject {subject}')
    pd_result, cv_result = cross_validate(subject)
    results[subject] = pd_result
    cv_results[subject] = cv_result

    

In [38]:
commutative = []
for subject in subjects:
    print(subject)
    all = []
    for session in cv_results[subject]:
        #print(session)
        if subject in session:
            #print(cv_results[subject][session])
            temp = cv_results[subject][session]
            
            zeros = np.zeros((4,4), dtype = np.float32)
            zeros[:temp.shape[0], :temp.shape[1]] = temp
            #temp.resize(4,4)
            all.append(zeros)
    all = np.array(all)
    summ = np.sum(all,axis = 0 )
    summ = np.array(summ, dtype = np.int)
    commutative.append(summ)
    #print(summ)
    #np.savetxt('confusion/SVM/'+subject+'.csv', summ,fmt='%i', delimiter=',')
        
commutative = np.array(commutative)
commutative =np.sum(commutative, axis = 0)
np.savetxt('confusion/SVM/commutative.csv', commutative,fmt='%i', delimiter=',')



S001
S002
S004
S005
S006
S007
S008
S009
S010
S011
S101
S102
S103
S104
S105
S106


C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
final = []
for d in results:
    by_subject = results[d].groupby(['Subject']).mean()
    final.append(by_subject.loc[:,'Accuracy'].tolist())
print(np.array(final))

In [ ]:
pd.concat([results['S001'], results['S002']])
asddd = [results['S001'].loc[:,'Accuracy'] + results['S002'].loc[:,'Accuracy']]
asddd

In [8]:
def cross_validate(subject, log = False):
    # use session4 for validation
    subject_to_val = [subject]
    train_sessions = []
    val_sessions = []

    for subject in os.listdir(root_path):
        for session in os.listdir(os.path.join(root_path,subject)):
            val_sessions.append(os.path.join(root_path,subject, session))
            if not subject in subject_to_val:
                train_sessions.append(os.path.join(root_path,subject, session))
        


    train_records = {}
    for c in classes:
        class_data = []
        for session in train_sessions:
            data = np.load(os.path.join(session,c+post_fix+'.npy'),allow_pickle=True)
            if data.shape[0] != 0:
                class_data.append(data)
            #else:
            #    print(f"No data available for train for class {c}")
        
        train_records[c] = np.concatenate(class_data)
    print(f"{len(train_sessions)} sessions loaded for training")


    val_records = {}
    for c in classes:
        class_data = []
        for session in val_sessions:
            data = np.load(os.path.join(session,c+post_fix+'.npy'),allow_pickle=True)
            if data.shape[0] != 0:
                class_data.append(data)
        if len(class_data) != 0:
            val_records[c] = np.concatenate(class_data)
        #else:
        #    print(f"No data available for validation for class {c}")

    print(f"{len(val_sessions)} sessions loaded for validation")





    n_channels = train_records["Rest"].shape[1]
    input_length = train_records["Rest"].shape[2]

    train_y = create_labels(train_records)
    train_X = np.concatenate((list(train_records.values())), axis=0)

    train_X, train_y = apply_augment(train_X, train_y)
    train_X = train_X.reshape(train_X.shape[0], n_channels*input_length)

    # param_grid = {'C': [10], 'gamma': [0.01]}

    # grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
    # grid.fit(train_X,train_y)
    # model = grid.best_estimator_ 
    svm = SVC(C=10, gamma=0.01)
    svm.fit(train_X, train_y)
    return evaluate_set(svm, val_sessions, classes, post_fix, log = log)
